In [1]:
import os

In [2]:
logging_dir = '.'
os.path.join(logging_dir, "/run")

'/run'

In [3]:
f"{logging_dir}/run"

'./run'

In [42]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import json

In [105]:
with open("round_robin/DR_FSP_DR_FSP.json") as f:
    logs = json.load(f)

In [106]:
pd.DataFrame(logs["LasertagArena1"])

,DR_FSP_2000_seed_1,DR_FSP_4000_seed_1,DR_FSP_6000_seed_1
DR_FSP_2000_seed_1,0.000000,0.000000,0.000000
DR_FSP_4000_seed_1,0.000000,0.000000,0.000000
DR_FSP_6000_seed_1,-0.333333,0.333333,-0.333333


In [108]:
px.imshow(pd.DataFrame(logs["LasertagArena2"]))